In [37]:
from pymongo import MongoClient
## pprint library is used to make the ouput more pretty
from pprint import pprint
#connect to MongoDB, change the << MONGODB URL >> to reflect your own connection string
client = MongoClient()
client 

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [38]:
db = client.basededonnee
collection = db.test_collection
db
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'basededonnee'), 'test_collection')

In [39]:
#kaggle dataset pour importer le csv de mails
import kaggle
kaggle.api.authenticate()
kaggle.api.dataset_download_files('wcukierski/enron-email-dataset')

## loading the data

In [40]:
#essai de voir les datas et les mettre dans un dataframe
import pandas as pd

In [41]:
df = pd.read_csv('enron-email-dataset/emails.csv')
df.shape

(517401, 2)

In [54]:
#explore one message
df.head(5)
print(df['message'][3])

Message-ID: <13505866.1075863688222.JavaMail.evans@thyme>
Date: Mon, 23 Oct 2000 06:13:00 -0700 (PDT)
From: phillip.allen@enron.com
To: randall.gay@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Randall L Gay
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

Randy,

 Can you send me a schedule of the salary and level of everyone in the 
scheduling group.  Plus your thoughts on any changes that need to be made.  
(Patti S for example)

Phillip


In [58]:
pd.set_option('display.max_columns',len(df.columns))
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [61]:
## missing values or duplicate
df.isnull().count()
df.duplicated().count()

517401

## cleaning


In [78]:

#def get_text(Series, row_num_slicer):
#"return a serie with text sliced from a list split"
result = df.message
print(result)
#split the message when we go to an other line
for row , message in enumerate(df.message):
    message_words = message.split('\n')
    #print(message_words)
    #my message is splitted
    #print(message_words[2])
print(message_words[1])
    

0         Message-ID: <18782981.1075855378110.JavaMail.e...
1         Message-ID: <15464986.1075855378456.JavaMail.e...
2         Message-ID: <24216240.1075855687451.JavaMail.e...
3         Message-ID: <13505866.1075863688222.JavaMail.e...
4         Message-ID: <30922949.1075863688243.JavaMail.e...
                                ...                        
517396    Message-ID: <26807948.1075842029936.JavaMail.e...
517397    Message-ID: <25835861.1075842029959.JavaMail.e...
517398    Message-ID: <28979867.1075842029988.JavaMail.e...
517399    Message-ID: <22052556.1075842030013.JavaMail.e...
517400    Message-ID: <28618979.1075842030037.JavaMail.e...
Name: message, Length: 517401, dtype: object
Date: Mon, 26 Nov 2001 10:48:43 -0800 (PST)


In [85]:
## make a function that get only the text
def get_text(Series):
    
    """
    Series: df series
    Row_num_slicer: number of row that we have sliced
    return a Serie with text sliced that we split for each message
    
    """
    result = pd.Series()
    for row, message in enumerate(Series):
        message_words = message.split('\n')
        
    return message_words
        
    

In [96]:
#separate info and content parts 
def info_part(i):
    """
    split the message in 2 part 
    the maxsplit parameter to 1, will return a list with 2 elements
    return the info text parsing 
    """
    return i.split('\n\n', 1)[0]


In [97]:
def content_part(i):
    """
    return the content part out
    """
    return i.split('\n\n',1)[1]

### new method

In [113]:
import re

In [108]:
emaildf= pd.read_csv('enron-email-dataset/emails.csv')

In [110]:
emaildf.shape

(517401, 2)

In [111]:
#get metadata headers
#je split en 2 
headers = [email.split(': ', 1)[0] for email in emaildf.iloc[0]['message'].split('\n')[:15]]

In [112]:
print(headers)

['Message-ID', 'Date', 'From', 'To', 'Subject', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding', 'X-From', 'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName']


In [114]:
#create a new column for every meta-data
for header in headers:
    emaildf[header]= emaildf.apply(lambda email : re.findall(rf'{header}:(.*)\n',email['message'])[0], axis=1) 

In [127]:
emaildf.head()

,file,...,X-FileName
0,allen-p/_sent_mail/1.,...,pallen (Non-Privileged).pst
1,allen-p/_sent_mail/10.,...,pallen (Non-Privileged).pst
2,allen-p/_sent_mail/100.,...,pallen.nsf
3,allen-p/_sent_mail/1000.,...,pallen.nsf
4,allen-p/_sent_mail/1001.,...,pallen.nsf


In [129]:
#add email's content to a new column
emaildf['Body']= emaildf.apply(lambda email : ''.join(re.findall('\n\n(.*)', email['message'])),axis=1)

In [131]:
emaildf['Body'][1]

"Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the presenter speaks and the others are quiet just waiting for their turn.   The meetings might be better if held in a round table discussion format.  My suggestion for where to go is Austin.  Play golf and rent a ski boat and jet ski's.  Flying somewhere takes too much time."

In [132]:
#now that we have a column for eachmetadat drop the "message" column
emaildf.drop(['message'], axis=1, inplace = True)

## put data on a database

In [138]:
#put the datas on Mongodb
#connect to mongoDB
from pymongo import MongoClient
client = MongoClient(port=27017)

#create database/collection
db= client.basededonnee
db.emails.drop()
collection = db['eron']
#transform dataframe to dictionnary
records_=emaildf.to_dict(orient='records')
#add many records
collection.insert_many(records_)

In [139]:
#getting a single document
collection.find_one()

{'_id': ObjectId('5e85b619affa6f27d9071cfa'),
 'file': 'allen-p/_sent_mail/1.',
 'Message-ID': ' <18782981.1075855378110.JavaMail.evans@thyme>',
 'Date': ' Mon, 14 May 2001 16:39:00 -0700 (PDT)',
 'From': ' phillip.allen@enron.com',
 'To': ' tim.belden@enron.com',
 'Subject': ' ',
 'Mime-Version': ' 1.0',
 'Content-Type': ' text/plain; charset=us-ascii',
 'Content-Transfer-Encoding': ' 7bit',
 'X-From': ' Phillip K Allen',
 'X-To': ' Tim Belden <Tim Belden/Enron@EnronXGate>',
 'X-cc': ' ',
 'X-bcc': ' ',
 'X-Folder': " \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail",
 'X-Origin': ' Allen-P',
 'X-FileName': ' pallen (Non-Privileged).pst',
 'Body': 'Here is our forecast '}

## BAG OF WORDS

In [143]:
#make a subdataset with 3000 rows
new_df = emaildf.iloc[:3000]


In [144]:
new_df.shape

(3000, 17)

In [145]:
new_df.head(5)

,file,...,Body
0,allen-p/_sent_mail/1.,...,Here is our forecast
1,allen-p/_sent_mail/10.,...,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,...,test successful. way to go!!!
3,allen-p/_sent_mail/1000.,...,"Randy, Can you send me a schedule of the salar..."
4,allen-p/_sent_mail/1001.,...,Let's shoot for Tuesday at 11:45.


In [146]:
new_df.tail(5)

,file,...,Body
2995,allen-p/sent/76.,...,---------------------- Forwarded by Phillip K ...
2996,allen-p/sent/765.,...,"Randy, Can you send me a schedule of the salar..."
2997,allen-p/sent/766.,...,Let's shoot for Tuesday at 11:45.
2998,allen-p/sent/767.,...,"Greg, How about either next Tuesday or Thursda..."
2999,allen-p/sent/768.,...,Please cc the following distribution list with...


In [148]:
!pip install nltk


  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449911 sha256=02c707349dea71cef8fec43612584f583b0042741b69aee3b5fe99b44183d049
  Stored in directory: c:\users\formation\appdata\local\pip\cache\wheels\48\8b\7f\473521e0c731c6566d631b281f323842bbda9bd819eb9a3ead
Successfully built nltk


In [149]:
#import libraries for nlp
from nltk import word_tokenize


In [150]:
new_df['Body'][0]

'Here is our forecast '

In [151]:
firt_row = new_df['Body'][0]

In [152]:
firt_row.split()

['Here', 'is', 'our', 'forecast']

In [173]:
df_exped = new_df.groupby('From')

In [175]:
df_exped.From.head(5)

0        phillip.allen@enron.com
1        phillip.allen@enron.com
2        phillip.allen@enron.com
3        phillip.allen@enron.com
4        phillip.allen@enron.com
                  ...           
2120          k..allen@enron.com
2121          k..allen@enron.com
2122          k..allen@enron.com
2123          k..allen@enron.com
2369        pam.butler@enron.com
Name: From, Length: 431, dtype: object

In [153]:
#make a new colum with the content of mail splitted
new_df['Body_split']= new_df['Boder '].str.split()

C:\Users\Formation\Anaconda3\envs\mongodb\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [156]:
new_df['Body_split'].head(5)

0                            [Here, is, our, forecast]
1    [Traveling, to, have, a, business, meeting, ta...
2                  [test, successful., way, to, go!!!]
3    [Randy,, Can, you, send, me, a, schedule, of, ...
4             [Let's, shoot, for, Tuesday, at, 11:45.]
Name: Body_split, dtype: object

In [157]:
new_df.columns

Index(['file', 'Message-ID', 'Date', 'From', 'To', 'Subject', 'Mime-Version',
       'Content-Type', 'Content-Transfer-Encoding', 'X-From', 'X-To', 'X-cc',
       'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'Body', 'Body_split'],
      dtype='object')

In [171]:
#grouper par expediteur
df_expe= new_df.groupby(['From']).count()
df_expe.head(5)

,file,...,Body_split
From,,,
1.11913372.-2@multexinvestornetwork.com,3,...,3
adrianne.engler@enron.com,6,...,6
al.pollard@newpower.com,1,...,1
alyse.herasimchuk@enron.com,3,...,3
anchordesk_daily@anchordesk.zdlists.com,20,...,20


## refait à l'envers en faisant d'abord le regroupement par mail

In [177]:
from collections import Counter

In [178]:
new_df.columns

Index(['file', 'Message-ID', 'Date', 'From', 'To', 'Subject', 'Mime-Version',
       'Content-Type', 'Content-Transfer-Encoding', 'X-From', 'X-To', 'X-cc',
       'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'Body', 'Body_split'],
      dtype='object')

In [179]:
new_df.drop(['Body_split'], axis=1, inplace=True)

C:\Users\Formation\Anaconda3\envs\mongodb\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [181]:
new_df.columns

Index(['file', 'Message-ID', 'Date', 'From', 'To', 'Subject', 'Mime-Version',
       'Content-Type', 'Content-Transfer-Encoding', 'X-From', 'X-To', 'X-cc',
       'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'Body'],
      dtype='object')

In [187]:
#new table with all words on emails joined
g=new_df.groupby('From')['Body'].agg(all_email = lambda x:''.join(x))

In [188]:
#print(g)

                                                                                  all_email
From                                                                                       
 1.11913372.-2@multexinvestornetwork.com  In today's Daily Update you'll find free repor...
 adrianne.engler@enron.com                Hi Phillip and Mike!Phillip/Mike - Phillip/Mik...
 al.pollard@newpower.com                  What is going on down the street?  Curious to ...
 alyse.herasimchuk@enron.com               Dear Associate / Analyst Committee:The follow...
 anchordesk_daily@anchordesk.zdlists.com  _____________________DAVID COURSEY____________...
...                                                                                     ...
 winnerannouncements@info.iwon.com                                                         
 wise.counsel@lpl.com                     Greetings Phillip,I received your phone messag...
 yahoo-delivers@yahoo-inc.com             [IMAGE]masthead=09 Yahoo!=20May 2001Ne

In [189]:
#split the words 
body_splitted= g['all_email'].str.split()

In [190]:
print(body_splitted)

From
 1.11913372.-2@multexinvestornetwork.com    [In, today's, Daily, Update, you'll, find, fre...
 adrianne.engler@enron.com                  [Hi, Phillip, and, Mike!Phillip/Mike, -, Phill...
 al.pollard@newpower.com                    [What, is, going, on, down, the, street?, Curi...
 alyse.herasimchuk@enron.com                [Dear, Associate, /, Analyst, Committee:The, f...
 anchordesk_daily@anchordesk.zdlists.com    [_____________________DAVID, COURSEY__________...
                                                                  ...                        
 winnerannouncements@info.iwon.com                                                         []
 wise.counsel@lpl.com                       [Greetings, Phillip,I, received, your, phone, ...
 yahoo-delivers@yahoo-inc.com               [[IMAGE]masthead=09, Yahoo!=20May, 2001New, Fe...
 yevgeny.frolov@enron.com                   [As, discussed, early, today, we, hereby, noti...
 yild@zdemail.zdlists.com                   [Y-Life, Da

In [194]:
#dictionnaire avec les mots les plus fréquents utilisés pour chaque email
word_freq={}
for exp in g.index:
    word_freq['%s'%exp]=Counter(body_splitted[exp]).most_common(3)

In [195]:
word_freq

{' 1.11913372.-2@multexinvestornetwork.com': [('to', 24),
  ('the', 24),
  ('of', 21)],
 ' adrianne.engler@enron.com': [('Phillip', 2), ('-', 2), ('Hi', 1)],
 ' al.pollard@newpower.com': [('you', 3), ('is', 2), ('going', 2)],
 ' alyse.herasimchuk@enron.com': [('/', 6), ('Analyst', 6), ('Dear', 3)],
 ' anchordesk_daily@anchordesk.zdlists.com': [('the', 81),
  ('to', 70),
  ('a', 45)],
 ' announce@inbox.nytimes.com': [('the', 45), ('a', 18), ('and', 15)],
 ' announcements.enron@enron.com': [('and', 8), ('to', 7), ('the', 6)],
 ' annualconference@prosrm.com': [('of', 14), ('and', 13), ('to', 12)],
 ' aod@newsdata.com': [('a', 16), ('this', 12), ('----', 10)],
 ' apkpcp@prodigy.net': [('give', 1), ('me', 1), ('your', 1)],
 ' arsystem@mailman.enron.com': [('this', 42), ('Please', 38), ('click', 38)],
 ' ashley.worthing@enron.com': [('report', 2), ('named:', 2), ('West', 2)],
 ' biliana.pehlivanova@enron.com': [('desk', 2), ('There', 1), ('are', 1)],
 ' billc@greenbuilder.com': [('other', 2)